# Lab 2 - Creating Entanglement with Qiskit

This lab demonstrates interesting properties of *entangled* qubits. In particular, we will consider two experiments:
- **CHSH Inequality Violation** - this shows that quantum mechanics *cannot* be explained by a local hidden variable theory
- **Teleportation** - teleport an arbitrary quantum state using an entangled qubit pair as a resource

In particular, this lab demonstrates how to use new features from IBM Quantum
- **Primitives** - abstract measurement and error mitigation for scalable quantum computing
- **Dynamic Circuits** - mid-circuit measurement and feed-forward within the qubits' coherence time

## Getting Started

Start by importing some libraries we need, including the `Sampler` and `Estimator` primitives from Qiskit. While the primitives from `qiskit.providers` use a local statevector simulator by default, the syntax within this lab is easily generalizable to running experiments on real systems.

To run on real hearware requires a Qiskit Runtime service instance. If you haven't done so already, follow the instructions in the Qiskit [Getting started guide](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/getting_started.html) to set one up. TODO: include video links and such. After setup, import the `Sampler` and `Estimator` primitives from `qiskit_ibm_runtime` instead. Additionally we will need `QiskitRuntimeService` and `Session`, which form the interface between Qiskit and Qiskit IBM Runtime. Then the below exercises can be run on real systems by instantiating the primitives in this way (as opposed to from `qiskit.primitives`):

```
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler, Estimator

service = QiskitRuntimeService()
backend = service.get_backend('...')
session = Session(service=service, backend=backend)
sampler = Sampler(session=session)
estimator = Estimator(session=session)
```
where additional options can be specified in the `Sampler` and `Estimator` with the `Options` class. See this [how-to](https://qiskit.org/ecosystem/ibm-runtime/how_to/run_session.html) for using Primitives with Runtime Sessions.


In [1]:
from qiskit.circuit import QuantumCircuit
from qiskit.primitives import Estimator, Sampler
from qiskit.quantum_info import SparsePauliOp
from qiskit.visualization import plot_histogram
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background') # optional